In [1]:
import os
import pickle
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import csv
# import math
# from itertools import groupby

# import eulpom.stochastic_modeling.utilities_markov_chain_training as umc
import utilities_markov_chain_training as umc
import utilities_markov_chain_simulation_validation as ums

In [2]:
act_data_name = os.path.join(os.path.dirname(os.path.dirname('__file__')), 'Data/raw_data/atusact-2021/atusact_2021.dat')
resp_data_name = os.path.join(os.path.dirname(os.path.dirname('__file__')), 'Data/raw_data/atusresp-2021/atusresp_2021.dat')
rost_data_name = os.path.join(os.path.dirname(os.path.dirname('__file__')), 'Data/raw_data/atusrost-2021/atusrost_2021.dat')

timeres = 15

In [3]:
pickle_name = os.path.join(os.path.dirname(os.path.dirname('__file__')), 'Data/all_atus_data_2021_TestProcessing.pickle')
loaded_pickle = umc.pickle_load(pickle_name)

In [4]:
clustering_algorithm_name = 'kmodes'

weekday_or_weekend = 'Weekday'
# weekday_or_weekend = 'Weekend'
weekday_or_weekend_for_input = [weekday_or_weekend]
# weekday_or_weekend = 'Weekend'

# one category for all data
ageRanges = ['18-120']
LiveAlones = ['LiveAloneYesNo']
childrenCategories = ['childrenYesNo']
employmentCategories = ['UnSemiFullyEmployed']
gender = ['12']

#state = 36
#county = 85
#countyname = 'RICHMOND'

idx_division_for_morning_midday_evening = [32, 64, 96]

physical_seg_name = '18-120LiveAloneYesNochildrenYesNoUnSemiFullyEmployed1'

physical_seg_name_old_version = '18-120childrenYesNoUnSemiFullyEmployed1'
physical_seg_name_old_version_special = '18-120childrenYesNoUnSemiFullyEmployed12'


In [4]:
len(loaded_pickle)


9087

In [8]:
#### 组合及normal#这一步是选取了原预处理之后的数据中想要的part（如weekday），选取之后len
num_respondants = len(loaded_pickle)
#len(loaded_pickle) = 8782
respondent_list_group_by_weekday_weekend = []
idx_respondent_reference = []
for i in range(num_respondants):
    extracted_record = loaded_pickle[i]
    # for i in range(1000):
    
#    listi = [extracted_record.dis1,extracted_record.dis2,extracted_record.dis3,
#            extracted_record.dis4,extracted_record.dis5,extracted_record.dis6]
    
    if extracted_record.day_type != weekday_or_weekend:
        continue

##  physical
 
#     elif extracted_record.age < 65:
#         continue
    
#     elif (1 not in listi):
#         continue
        
### Social economic

# ##    #income
#     elif extracted_record.fmember <=1 and extracted_record.income >=5:
#         continue
#     elif extracted_record.fmember <=2 and extracted_record.income >=6:
#         continue    
#     elif extracted_record.fmember <=3 and extracted_record.income >=7:
#         continue 
#     elif extracted_record.fmember <=4 and extracted_record.income >=8:
#         continue 
#     elif extracted_record.fmember <=5 and extracted_record.income >=9:
#         continue 
#     elif extracted_record.fmember <=6 and extracted_record.income >=10:
#         continue    
#     elif extracted_record.fmember <=8 and extracted_record.income >=11:
#         continue 
#     elif extracted_record.fmember <=9 and extracted_record.income >=12:
#         continue 
#     elif extracted_record.fmember <=10 and extracted_record.income >=13:
#         continue 
#     elif extracted_record.fmember >= 14:
#         continue 



#######  workstate
#     elif extracted_record.work == 1 or extracted_record.work == -1 or extracted_record.work == 3 or extracted_record.work == 4 :
#         continue

# ######## education
#     elif extracted_record.education >= 40:
#         continue

######## minority
    elif extracted_record.minority == 1:
        continue

# # ######## foreign
#     elif extracted_record.foreign != 5:
#         continue

#     elif extracted_record.age >= 65:
#         continue
    
#     elif (1 not in listi):
#         continue
        
#     elif extracted_record.dis2 != 1:
#         continue
        
#     elif extracted_record.county != county:
#         continue  
      
    if not extracted_record.occupancy_data_too_short:
        idx_respondent_reference.append(i)
        respondent_list_group_by_weekday_weekend.append(extracted_record)
cluster_label = []
all_idx_this_physical_seg = list(range(len(respondent_list_group_by_weekday_weekend)))

respondent_list_for_this_segmentation_further_selected = []
for i in range(len(all_idx_this_physical_seg)):
    respondent_list_for_this_segmentation_further_selected.append(
        respondent_list_group_by_weekday_weekend[all_idx_this_physical_seg[i]])


In [9]:
len(respondent_list_for_this_segmentation_further_selected)


4077

In [10]:
activityIDToIndexDict = {'010501':0, '010301':0, '080402':0,# 101 Personal Health_Residential
                         '080401':1, '080403':1, # 102 Personal Health_Public Service
                         '010101':2, '010102':2, '010201':2, '010401':2, # 103 Biological_Residential
                         '080501':3, '080502':3, # 104 Biological_Business
                         '020101':4,'020102':4,'020103':4,'020104':4,'020201':4,'020202':4,'020203':4,'020301':4,
                         '020302':4,'020303':4,'020401':4,'020402':4,'020501':4,'020502':4,'020601':4,'020602':4,
                         '020701':4,'020801':4,'020901':4,'020902':4,'020903':4,'020904':4,'020905':4,'090101':4,
                         '090102':4,'090103':4,'090104':4,'090201':4,'090202':4,'090301':4,'090302':4,'090401':4,
                         '090402':4,'090501':4,'090502':4, # 105 Household duties
                         '030101':5,'030102':5,'030103':5,'030104':5,'030106':5,'030108':5,'030109':5,
                         '030201':5,'030203':5,'030301':5,'030302':5,'030303':5,'030401':5,'030402':5, 
                         '030403':5,'030404':5,'030405':5,'030502':5,'040101':5,'040102':5,'040103':5, 
                         '040104':5,'040106':5,'040108':5,'040109':5,'040201':5,
                         '040203':5,'040301':5,'040302':5,'040303':5,'040401':5, '040402':5, 
                         '040403':5,'040404':5,'040405':5,'040501':5,'040502':5, '040503':5, '040504':5, 
                         '040505':5,'040506':5,'160101':5,'160102':5,'160106':5, '160107':5, # 106 obligation_RES
                         '030110':6,'030111':6,'030202':6,'030204':6,'040110':6,'040111':6,'040202':6,
                         '040204':6,'160103':6, #107 Obligation_EDU
                         '080201':7,'080202':7,'080203':7,'080301':7, '080302':7,'080601':7, '080602':7,
                         '080701':7,'080702':7,'080801':7,'160108':7, #108 Obli_Busi
                         '070101':8,'070102':8,'070103':8,'070104':8,'070105':8,'070201':8,'070301':8,'160104':8, #109 Pbli_Merch
                         '070102':9,'080101':9,'080102':9,'100101':9,'100102':9, '100103':9, '100201':9,
                         '100304':9,'100305':9,'100401':9,'160105':9, #110 Obli_Pub
                         '030105':10,'030112':10,'030503':10,'030504':10,'040105':10,'040112':10,
                         '040507':10,'040508':10,  #111 Obli_Out
                         '050101':11, '050102':11, '050103':11, '050104':11,'050201':11, '050202':11, '050203':11, 
                         '050204':11, '050205':11, '050301':11, '050302':11, '050303':11,'050304':11, '050305':11, #112 Work_Occ
                         '050403':11,'050404':11, '050405':11, # 113 Working_Busi
                         '060101':13, '060102':13, '060103':13, '060104':13, '060401':13, '060402':13, '060403':13,
                         '060201':13, '060202':13, '060203':13, '060204':13, #114Edu_Edu
                         '060301':14, '060302':14, '060303':14, #115 EDu_Resi
                        # '060201':15, '060202':15, '060203':15, '060204':15,  #116 Edu_Assem
                         '120201':16, '120303':16, '120304':16,'120305':16, '120306':16,'120307':16, '120308':16,
                         '120309':16, '120310':16, '120312':16, '120313':16,'130101':16, #117 Pref_Resi
                         '120401':17, '120402':17,'120403':17, '120404':17,'120405':17,
                         '120501':17, '120502':17,'120503':17, '120504':17,'130107':17, '130120':17, '130121':17,
                         '130122':17, '130128':17, '130133':17, '130134':17, '130135':17, '130136':17,'130202':17,
                         '130203':17, '130204':17, '130205':17,'130206':17, '130207':17, '130209':17,'130210':17, 
                         '130211':17, '130213':17, '130215':17, '130216':17,'130217':17, '130218':17, '130219':17,
                         '130220':17, '130221':17, '130223':17,'130224':17, '130225':17, '130226':17,'130227':17, 
                         '130229':17, '130230':17, '130232':17,'130302':17,'130401':17, '130402':17,'140101':17,'140102':17, 
                         '140103':17, '140104':17, '140105':17,'150204':17, '150401':17, '150402':17,'150801':17,#118 Pre_Ass
                         '120311':18, '130102':18, '130103':18,'130104':18,'130105':18,'130106':18,
                         '130108':18, '130109':18,'130110':18, '130111':18, '130112':18,'130113':18, '130114':18,
                         '130115':18, '130116':18,'130117':18, '130118':18, '130119':18,'130123':18, '130124':18,
                         '130125':18, '130126':18,'130127':18,'130129':18,'130130':18,'130131':18,'130132':18,
                         '130201':18,'130208':18,'130212':18,'130214':18,'130222':18,'130228':18,'130231':18,
                         '130301':18,'150202':18,'150301':18,'150601':18,'150602':18,
                         '180101':18, '180201':18,'180202':18, '180203':18, '180204':18, '180205':18, '180206':18,
                         '180207':18, '180208':18,'180209':18, '180301':18, '180302':18, 
                         '180303':18, '180304':18, '180305':18, '180401':18,'180402':18, '180403':18, '180404':18,
                         '180405':18, '180501':18,'180502':18, '180503':18, '180504':18, '180601':18,'180602':18,
                         '180603':18, '180604':18,'180701':18,'180702':18, '180703':18, '180704':18, '180801':18,
                         '180802':18, '180803':18, '180804':18,'180805':18, '180806':18, '180807':18,'180901':18,
                         '180902':18, '180903':18, '180904':18,'180905':18, '181001':18,'181002':18, '181101':18,
                         '181201':18, '181202':18, '181203':18, '181204':18,'181205':18,'181301':18,'181302':18,
                         '181401':18, '181501':18, '181601':18, '181801':18, #119 Pref_Out
                         '030501':19,'050401':19,'110101':19, '110201':19,'110101':19, '110201':19,'110301':19,
                         '120302':19,'150101':19,'150102':19, '150103':19, 
                         '150104':19, '150105':19, '150106':19, '150201':19,'150203':19,'150302':19,'150501':19,
                         '150701':19,'160201':19}  # 120 Anywhere


corresponding_actVals = {'010501', '010301', '080402',# 101 Personal Health_Residential
                         '080401', '080403', # 102 Personal Health_Public Service
                         '010101', '010102','010201', '010401', # 103 Biological_Residential
                         '080501', '080502', # 104 Biological_Business
                         '020101','020102','020103','020104','020201','020202','020203','020301',
                         '020302','020303','020401','020402','020501','020502','020601','020602',
                         '020701','020801','020901','020902','020903','020904','020905','090101',
                         '090102','090103','090104','090201','090202','090301','090302','090401',
                         '090402','090501','090502', # 105 Household duties
                         '030101','030102','030103','030104','030106','030108','030109',
                         '030201','030203','030301','030302','030303','030401','030402', 
                         '030403','030404','030405','030502','040101','040102','040103', 
                         '040104','040106','040108','040109','040201',
                         '040203','040301','040302','040303','040401', '040402', 
                         '040403','040404','040405','040501','040502', '040503', '040504', 
                         '040505','040506','160101','160102','160106', '160107', # 106 obligation_RES
                         '030110','030111','030202','030204','040110','040111','040202',
                         '040204','160103', #107 Obligation_EDU
                         '080201','080202','080203','080301','080302','080601', '080602',
                         '080701','080702','080801','160108', #108 Obli_Busi
                         '070101','070102','070103','070104','070105','070201','070301','160104', #109 Pbli_Merch
                         '070102','080101','080102','100101','100102', '100103', '100201',
                         '100304','100305','100401','160105', #110 Obli_Pub
                         '030105','030112','030503','030504','040105','040112',
                         '040507','040508',  #111 Obli_Out
                         '050101', '050102', '050103', '050104','050201', '050202', '050203', 
                         '050204', '050205', '050301', '050302', '050303','050304', '050305', 
                         '050403','050404', '050405',#112 Work_Occ
                         #'050403','050404', '050405', # 113 Working_Busi
                         '060101', '060102', '060103', '060104', '060401', '060402', '060403',
                         '060201', '060202', '060203', '060204', # 114Edu_Edu
                         '060301', '060302', '060303', #115 EDu_Resi
                         #'060201', '060202', '060203', '060204',  #116 Edu_Assem
                         '120201', '120303', '120304','120305', '120306','120307', '120308',
                         '120309', '120310', '120312', '120313','130101', #117 Pref_Resi
                         '120401', '120402','120403', '120404','120405',
                         '120501', '120502','120503', '120504','130107', '130120', '130121',
                         '130122', '130128', '130133', '130134', '130135', '130136','130202',
                         '130203', '130204', '130205','130206', '130207', '130209','130210', 
                         '130211', '130213', '130215', '130216','130217', '130218', '130219',
                         '130220', '130221', '130223','130224', '130225', '130226','130227', 
                         '130229', '130230', '130232','130302','130401', '130402','140101','140102', 
                         '140103', '140104', '140105','150204', '150401', '150402','150801',#118 Pre_Ass
                         '120311', '130102', '130103','130104','130105','130106',
                         '130108', '130109','130110', '130111', '130112','130113', '130114',
                         '130115', '130116','130117', '130118', '130119','130123', '130124',
                         '130125', '130126','130127','130129','130130','130131','130132',
                         '130201','130208','130212','130214','130222','130228','130231',
                         '130301','150202','150301','150601','150602',
                         '180101', '180201','180202', '180203', '180204', '180205', '180206',
                         '180207', '180208','180209', '180301', '180302', 
                         '180303', '180304', '180305', '180401','180402', '180403', '180404',
                         '180405', '180501','180502', '180503', '180504', '180601','180602',
                         '180603', '180604','180701','180702', '180703', '180704', '180801',
                         '180802', '180803', '180804','180805', '180806', '180807','180901',
                         '180902', '180903', '180904','180905', '181001','181002', '181101',
                         '181201', '181202', '181203', '181204','181205','181301','181302',
                         '181401', '181501', '181601', '181801', #119 Pref_Out
                         '030501','050401','110101', '110201','110101', '110201','110301',
                         '120302','150101','150102', '150103', 
                         '150104', '150105', '150106', '150201','150203','150302','150501',
                         '150701','160201'}  # 120 Anywhere
num_status_in_markov_sim = 20
indicator_to_turn_on_clustering = 1

In [11]:
  # ///////////////////////////////// MC activity data processing part - duration //////////////////////////////////

c101= ['010501', '010301', '080402']
c102=['080401', '080403'] # 102 Personal Health_Public Service
c103=['010101', '010102','010201', '010401'] # 103 Biological_Residential
c104=['080501', '080502'] # 104 Biological_Business
c105=['020101','020102','020103','020104','020201','020202','020203','020301',
      '020302','020303','020401','020402','020501','020502','020601','020602',
      '020701','020801','020901','020902','020903','020904','020905','090101',
      '090102','090103','090104','090201','090202','090301','090302','090401',
      '090402','090501','090502'] # 105 Household duties
c106=['030101','030102','030103','030104','030106','030108','030109',
      '030201','030203','030301','030302','030303','030401','030402', 
      '030403','030404','030405','030502','040101','040102','040103', 
      '040104','040106','040108','040109','040201',
      '040203','040301','040302','040303','040401', '040402', 
      '040403','040404','040405','040501','040502', '040503', '040504', 
      '040505','040506','160101','160102','160106', '160107'] # 106 obligation_RES
c107=['030110','030111','030202','030204','040110','040111','040202','040204','160103'] #107 Obligation_EDU
c108=['080201','080202','080203','080301','080302','080601', '080602','080701','080702','080801','160108'] #108 Obli_Busi
c109=['070101','070102','070103','070104','070105','070201','070301','160104']#109 Pbli_Merch
c110=['070102','080101','080102','100101','100102', '100103', '100201','100304','100305','100401','160105'] #110 Obli_Pub
c111=['030105','030112','030503','030504','040105','040112','040507','040508']  #111 Obli_Out
c112=['050101', '050102', '050103', '050104','050201', '050202', '050203', '050204', '050205',
      '050301', '050302', '050303','050304', '050305','050403','050404', '050405'] #112 Work_Occ
#c113=['050403','050404', '050405'] # 113 Working_Busi
c114=['060101', '060102', '060103', '060104', '060401', '060402', '060403','060201','060202', '060203', '060204']# 114Edu_Edu
c115=['060301', '060302', '060303'] #115 EDu_Resi
# c116=['060201', '060202', '060203', '060204'] #116 Edu_Assem
c117=['120201', '120303', '120304','120305', '120306','120307', '120308',
      '120309', '120310', '120312', '120313','130101'] #117 Pref_Resi
c118=['120401', '120402','120403', '120404','120405',
      '120501', '120502','120503', '120504','130107', '130120', '130121',
      '130122', '130128', '130133', '130134', '130135', '130136','130202',
      '130203', '130204', '130205','130206', '130207', '130209','130210', 
      '130211', '130213', '130215', '130216','130217', '130218', '130219',
      '130220', '130221', '130223','130224', '130225', '130226','130227', 
      '130229', '130230', '130232','130302','130401', '130402','140101','140102', 
      '140103', '140104', '140105','150204', '150401', '150402','150801']#118 Pre_Ass
c119=['120311', '130102', '130103','130104','130105','130106',
      '130108', '130109','130110', '130111', '130112','130113', '130114',
      '130115', '130116','130117', '130118', '130119','130123', '130124',
      '130125', '130126','130127','130129','130130','130131','130132',
      '130201', '130208','130212','130214','130222','130228','130231',
      '130301', '150202','150301','150601','150602',
      '180101', '180201','180202', '180203', '180204', '180205', '180206',
      '180207', '180208','180209', '180301', '180302', 
      '180303', '180304', '180305', '180401','180402', '180403', '180404',
      '180405', '180501','180502', '180503', '180504', '180601','180602',
      '180603', '180604','180701','180702', '180703', '180704', '180801',
      '180802', '180803', '180804','180805', '180806', '180807','180901',
      '180902', '180903', '180904','180905', '181001','181002', '181101',
      '181201', '181202', '181203', '181204','181205','181301','181302',
      '181401', '181501', '181601', '181801'] #119 Pref_Out
c120=['030501','050401','110101', '110201','110101', '110201','110301',
      '120302','150101','150102', '150103', 
      '150104', '150105', '150106', '150201','150203','150302','150501','150701','160201']  # 120 Anywhere

# readingCodes = ['120312', '150103']
# eatingCodes = ['110101']
# cleaningCodes = ['020101']

codeDict = {}
codeDict['101c'] = c101
codeDict['102c'] = c102
codeDict['103c'] = c103
codeDict['104c'] = c104
codeDict['105c'] = c105
codeDict['106c'] = c106
codeDict['107c'] = c107
codeDict['108c'] = c108
codeDict['109c'] = c109
codeDict['110c'] = c110      
codeDict['111c'] = c111
codeDict['112c'] = c112
# codeDict['113c'] = c113
codeDict['114c'] = c114
codeDict['115c'] = c115
# codeDict['116c'] = c116
codeDict['117c'] = c117
codeDict['118c'] = c118
codeDict['119c'] = c119
codeDict['120c'] = c120
# codeDict['reading'] = readingCodes
# codeDict['eating'] = eatingCodes
# codeDict['cleaning'] = cleaningCodes
trained_markov_sim_file_save_dir = os.path.join(os.path.dirname('__file__'), 'nominority'+'/'+ weekday_or_weekend + '/')
if not os.path.exists(trained_markov_sim_file_save_dir):
    os.makedirs(trained_markov_sim_file_save_dir)
activity_series_save_dir = trained_markov_sim_file_save_dir

In [12]:
# /////////////////////all data processing - can be commented out if processed /////////////////////////
# /////////////////// this part is process for data of one segmentation //////////////////
# old activityDict put weights for all timesteps one activity is ON in the duration,
# but new activityDict only put the weights on onset timestep, so new
# activityDict is for onset proability calculation
for activity in list(codeDict.keys()):
    print('data processing for ' + activity)

    # the activityDict here is the old dict
    # activityDict = activityData_per_group(atuscsv_data_name, actsumcsv_data_name,
    # respondent_list_for_this_segmentation_further_selected, codeDict[activity])
    # activityDict,weight_occ = umc.activityData_per_group_with_weight(atuscsv_data_name,
    # actsumcsv_data_name,respondent_list_for_this_segmentation_further_selected, codeDict[activity])
    activityDict, weight_occ = umc.activityData_per_group_with_weight(
        act_data_name, resp_data_name, respondent_list_for_this_segmentation_further_selected, codeDict[activity])

    # activityArray = np.array(list(activityDict.values()))
    #
    # activitySum = np.sum(activityArray, axis =0)
    # activitySum = activitySum/activityArray.shape[0]

    old_activity_data_save_name = weekday_or_weekend + 'nominority' +'activityData_' + activity + \
        '_Data_' + physical_seg_name + '_0319.pickle'
    


    try:
        with open(activity_series_save_dir + old_activity_data_save_name, 'wb') as picklefile:
            pickle.dump(activityDict, picklefile)
            pickle.dump(weight_occ, picklefile)
            print('old activity data ' + activity + ' dumping successful')
    except BaseException:
        print('old activity data ' + activity + ' dumping unsuccessful')
        

data processing for 101c
data processing for record num 1
data processing for record num 2
data processing for record num 3
data processing for record num 4
data processing for record num 5
data processing for record num 6
data processing for record num 7
data processing for record num 8
data processing for record num 9
data processing for record num 10
data processing for record num 11
data processing for record num 12
data processing for record num 13
data processing for record num 14
data processing for record num 15
data processing for record num 16
data processing for record num 17
data processing for record num 18
data processing for record num 19
data processing for record num 20
data processing for record num 21
data processing for record num 22
data processing for record num 23
data processing for record num 24
data processing for record num 25
data processing for record num 26
data processing for record num 27
data processing for record num 28
data processing for record num 2

data processing for record num 440
data processing for record num 441
data processing for record num 442
data processing for record num 443
data processing for record num 444
data processing for record num 445
data processing for record num 446
data processing for record num 447
data processing for record num 448
data processing for record num 449
data processing for record num 450
data processing for record num 451
data processing for record num 452
data processing for record num 453
data processing for record num 454
data processing for record num 455
data processing for record num 456
data processing for record num 457
data processing for record num 458
data processing for record num 459
data processing for record num 460
data processing for record num 461
data processing for record num 462
data processing for record num 463
data processing for record num 464
data processing for record num 465
data processing for record num 466
data processing for record num 467
data processing for 

data processing for record num 886
data processing for record num 887
data processing for record num 888
data processing for record num 889
data processing for record num 890
data processing for record num 891
data processing for record num 892
data processing for record num 893
data processing for record num 894
data processing for record num 895
data processing for record num 896
data processing for record num 897
data processing for record num 898
data processing for record num 899
data processing for record num 900
data processing for record num 901
data processing for record num 902
data processing for record num 903
data processing for record num 904
data processing for record num 905
data processing for record num 906
data processing for record num 907
data processing for record num 908
data processing for record num 909
data processing for record num 910
data processing for record num 911
data processing for record num 912
data processing for record num 913
data processing for 

data processing for record num 1117
data processing for record num 1118
data processing for record num 1119
data processing for record num 1120
data processing for record num 1121
data processing for record num 1122
data processing for record num 1123
data processing for record num 1124
data processing for record num 1125
data processing for record num 1126
data processing for record num 1127
data processing for record num 1128
data processing for record num 1129
data processing for record num 1130
data processing for record num 1131
data processing for record num 1132
data processing for record num 1133
data processing for record num 1134
data processing for record num 1135
data processing for record num 1136
data processing for record num 1137
data processing for record num 1138
data processing for record num 1139
data processing for record num 1140
data processing for record num 1141
data processing for record num 1142
data processing for record num 1143
data processing for record n

data processing for record num 1352
data processing for record num 1353
data processing for record num 1354
data processing for record num 1355
data processing for record num 1356
data processing for record num 1357
data processing for record num 1358
data processing for record num 1359
data processing for record num 1360
data processing for record num 1361
data processing for record num 1362
data processing for record num 1363
data processing for record num 1364
data processing for record num 1365
data processing for record num 1366
data processing for record num 1367
data processing for record num 1368
data processing for record num 1369
data processing for record num 1370
data processing for record num 1371
data processing for record num 1372
data processing for record num 1373
data processing for record num 1374
data processing for record num 1375
data processing for record num 1376
data processing for record num 1377
data processing for record num 1378
data processing for record n

...

data processing for record num 2771
data processing for record num 2772
data processing for record num 2773
data processing for record num 2774
data processing for record num 2775
data processing for record num 2776
data processing for record num 2777
data processing for record num 2778
data processing for record num 2779
data processing for record num 2780
data processing for record num 2781
data processing for record num 2782
data processing for record num 2783
data processing for record num 2784
data processing for record num 2785
data processing for record num 2786
data processing for record num 2787
data processing for record num 2788
data processing for record num 2789
data processing for record num 2790
data processing for record num 2791
data processing for record num 2792
data processing for record num 2793
data processing for record num 2794
data processing for record num 2795
data processing for record num 2796
data processing for record num 2797
data processing for record n

data processing for record num 2999
data processing for record num 3000
data processing for record num 3001
data processing for record num 3002
data processing for record num 3003
data processing for record num 3004
data processing for record num 3005
data processing for record num 3006
data processing for record num 3007
data processing for record num 3008
data processing for record num 3009
data processing for record num 3010
data processing for record num 3011
data processing for record num 3012
data processing for record num 3013
data processing for record num 3014
data processing for record num 3015
data processing for record num 3016
data processing for record num 3017
data processing for record num 3018
data processing for record num 3019
data processing for record num 3020
data processing for record num 3021
data processing for record num 3022
data processing for record num 3023
data processing for record num 3024
data processing for record num 3025
data processing for record n

data processing for record num 3457
data processing for record num 3458
data processing for record num 3459
data processing for record num 3460
data processing for record num 3461
data processing for record num 3462
data processing for record num 3463
data processing for record num 3464
data processing for record num 3465
data processing for record num 3466
data processing for record num 3467
data processing for record num 3468
data processing for record num 3469
data processing for record num 3470
data processing for record num 3471
data processing for record num 3472
data processing for record num 3473
data processing for record num 3474
data processing for record num 3475
data processing for record num 3476
data processing for record num 3477
data processing for record num 3478
data processing for record num 3479
data processing for record num 3480
data processing for record num 3481
data processing for record num 3482
data processing for record num 3483
data processing for record n

data processing for record num 3685
data processing for record num 3686
data processing for record num 3687
data processing for record num 3688
data processing for record num 3689
data processing for record num 3690
data processing for record num 3691
data processing for record num 3692
data processing for record num 3693
data processing for record num 3694
data processing for record num 3695
data processing for record num 3696
data processing for record num 3697
data processing for record num 3698
data processing for record num 3699
data processing for record num 3700
data processing for record num 3701
data processing for record num 3702
data processing for record num 3703
data processing for record num 3704
data processing for record num 3705
data processing for record num 3706
data processing for record num 3707
data processing for record num 3708
data processing for record num 3709
data processing for record num 3710
data processing for record num 3711
data processing for record n

data processing for record num 3917
data processing for record num 3918
data processing for record num 3919
data processing for record num 3920
data processing for record num 3921
data processing for record num 3922
data processing for record num 3923
data processing for record num 3924
data processing for record num 3925
data processing for record num 3926
data processing for record num 3927
data processing for record num 3928
data processing for record num 3929
data processing for record num 3930
data processing for record num 3931
data processing for record num 3932
data processing for record num 3933
data processing for record num 3934
data processing for record num 3935
data processing for record num 3936
data processing for record num 3937
data processing for record num 3938
data processing for record num 3939
data processing for record num 3940
data processing for record num 3941
data processing for record num 3942
data processing for record num 3943
data processing for record n

In [13]:
# new data
for activity in list(codeDict.keys()):

    old_activity_data_save_name = weekday_or_weekend +'nominority'+ 'activityData_' + activity + '_Data_' + \
        physical_seg_name + '_0319.pickle'

    try:
        with open(activity_series_save_dir + old_activity_data_save_name, 'rb') as picklefile:
            activityDict = pickle.load(picklefile)
            weight_occ = pickle.load(picklefile)
            print('old activity data loading successful')
    except BaseException:
        print('old activity data loading unsuccessful')

    print('new data processing for ' + activity)

    newActivityDict = umc.activityOnsetProbability(activityDict)
    # activityArray = np.array(list(newActivityDict.values()))

    # activitySum = np.sum(activityArray, axis=0)
    # activitySum = activitySum / activityArray.shape[0]

    new_activity_data_save_name = weekday_or_weekend + 'nominority' + 'activityData' + 'new_' + activity + '_Data' + \
        physical_seg_name + '_0319.pickle'

    try:
        with open(activity_series_save_dir + new_activity_data_save_name, 'wb') as picklefile:
            pickle.dump(newActivityDict, picklefile)
            pickle.dump(weight_occ, picklefile)
            print('new activity data ' + activity + ' dumping successful')
    except BaseException:
        print('new activity data ' + activity + ' dumping unsuccessful')
        

old activity data loading successful
new data processing for 101c
new activity data 101c dumping successful
old activity data loading successful
new data processing for 102c
new activity data 102c dumping successful
old activity data loading successful
new data processing for 103c
new activity data 103c dumping successful
old activity data loading successful
new data processing for 104c
new activity data 104c dumping successful
old activity data loading successful
new data processing for 105c
new activity data 105c dumping successful
old activity data loading successful
new data processing for 106c
new activity data 106c dumping successful
old activity data loading successful
new data processing for 107c
new activity data 107c dumping successful
old activity data loading successful
new data processing for 108c
new activity data 108c dumping successful
old activity data loading successful
new data processing for 109c
new activity data 109c dumping successful
old activity data loading su

In [14]:
activity_series_save_dir

'nominority/Weekday/'

In [15]:
oldActivityDict_list = []
newActivityDict_list = []
weight_occ_real_data_list = []
enduse_list = ['101c', '102c', '103c', '104c', '105c', '106c','107c','108c','109c','110c','111c','112c',
               '114c','115c','117c','118c','119c','120c']

# for enduse in ['shower','cleaning', 'cooking', 'TV', 'dishwashing','laundry', 'eating', 'reading']:
for enduse in enduse_list:

    # old_activity_data_save_name = weekday_or_weekend + 'activityData_' + enduse + '_Data_' +
    # physical_seg_name_old_version + '.pickle'
    old_activity_data_save_name = weekday_or_weekend + 'nominority'+'activityData_' + \
        enduse + '_Data_' + physical_seg_name + '_0319.pickle'
    try:
        with open(activity_series_save_dir + old_activity_data_save_name, 'rb') as picklefile:
            oldActivityDict = pickle.load(picklefile)
            weight_occ = pickle.load(picklefile)
            # print('old activity data loading successful')
    except BaseException:
        print('old activity data loading unsuccessful')

    oldActivityDict_list.append(oldActivityDict)

    # new_activity_data_save_name = weekday_or_weekend + 'activityData' + 'new_' + enduse + '_Data' +
    # physical_seg_name_old_version + '.pickle'
    new_activity_data_save_name = weekday_or_weekend + 'nominority'+ 'activityData' + \
        'new_' + enduse + '_Data' + physical_seg_name + '_0319.pickle'

    try:
        with open(activity_series_save_dir + new_activity_data_save_name, 'rb') as picklefile:
            activityDict = pickle.load(picklefile)
            weight_occ = pickle.load(picklefile)
            # print('new activity data loading successful')
    except BaseException:
        print('new activity data loading unsuccessful')

    newActivityDict_list.append(activityDict)
    weight_occ_real_data_list.append(weight_occ)

# from here, begin to generate statistics f

# enduse_list = ['shower', 'cleaning', 'cooking', 'TV', 'dishwashing', 'laundry', 'eating', 'reading']

num_event_prob_real_key_values_all_data_all_act = []
num_event_prob_real_all_data_all_act = []
event_dur_prob_real_key_values_all_data_all_act = []
event_dur_prob_real_all_data_all_act = []
onset_prob_real_all_data_all_act = []

num_event_prob_sim_key_values_all_data_all_act = []
num_event_prob_sim_all_data_all_act = []
event_dur_prob_sim_key_values_all_data_all_act = []
event_dur_prob_sim_all_data_all_act = []
onset_prob_sim_all_data_all_act = []




# fig, axs = plt.subplots(4, 4)



for m in range(len(enduse_list)):
    # activity_name_to_compare = 'shower'
    # activity_name_to_compare = 'cooking'
    # activity_name_to_compare = 'cleaning'
    # activity_name_to_compare = 'TV'
    # activity_name_to_compare = 'dishwashing'
    # activity_name_to_compare = 'laundry'

    activity_name_to_compare = enduse_list[m]

    # weight_occ_real = weight_occ
    weight_occ_real = weight_occ_real_data_list[m]
    # activityDict is new
    idx_in_end_use_list = enduse_list.index(activity_name_to_compare)
    activityDict = newActivityDict_list[idx_in_end_use_list]
    oldActivityDict = oldActivityDict_list[idx_in_end_use_list]

    # ///////// generate statistics for real data /////////

    count_number_activity_in_morning = 0
    count_number_activity_in_afternoon = 0
    count_number_activity_in_evening = 0
    count_no_activity = 0

    total_number_records = 0
    for ind in list(oldActivityDict.keys()):

        extracted_record = oldActivityDict[ind]

        if sum(extracted_record[:idx_division_for_morning_midday_evening[0]]) > 0:
            count_number_activity_in_morning = count_number_activity_in_morning + 1

        if sum(extracted_record[
               idx_division_for_morning_midday_evening[0]:idx_division_for_morning_midday_evening[1]]) \
                > 0:
            count_number_activity_in_afternoon = count_number_activity_in_afternoon + 1

        if sum(extracted_record[idx_division_for_morning_midday_evening[1]:]) > 0:
            count_number_activity_in_evening = count_number_activity_in_evening + 1

        if sum(extracted_record[idx_division_for_morning_midday_evening[1]:]) == 0:
            count_no_activity = count_no_activity + 1

        total_number_records = total_number_records + 1

    percent_morning = count_number_activity_in_morning / total_number_records
    percent_afternoon = count_number_activity_in_afternoon / total_number_records
    percent_evening = count_number_activity_in_evening / total_number_records
    percent_no_activity = count_no_activity / total_number_records

    print(
        'real probability on - percentage of ' +
        activity_name_to_compare +
        ' in the morning ' +
        str(percent_morning) +
        ' in the afternoon ' +
        str(percent_afternoon) +
        ' in the evening ' +
        str(percent_evening) +
        ' no activity ' +
        str(percent_no_activity))

    # eventProbs = ums.calculateNumberEventsProbability(activityDict,weight_occ_real)
    eventProbs = ums.calculateNumberEventsProbability(
        activityDict, weight_occ_real, idx_division_for_morning_midday_evening)

    keyvals = list(eventProbs.keys())
    keyvals = sorted(keyvals)
    probs_real_num_event = []
    for i in range(len(keyvals)):
        probs_real_num_event.append(eventProbs[keyvals[i]])

    # fig_save_name = activity_name_to_compare + '_number_of_event_probabilities_real'
    # plt.plot(keyvals, probs_real_num_event)
    # plt.title('real data - number of event probabilities')
    # # plt.show()
    # plt.savefig(fig_save_dir + fig_save_name)
    # plt.close('all')

    num_event_prob_real_key_values_all_data_all_act.append(keyvals)
    num_event_prob_real_all_data_all_act.append(probs_real_num_event)

    # durationProbs = calculateDurationProbability(oldActivityDict,weight_occ_real)
    durationProbs = ums.calculateDurationProbability_V2(
        oldActivityDict, weight_occ_real, idx_division_for_morning_midday_evening)
    # print(durationProbs)

    keyvals = list(durationProbs.keys())
    keyvals = sorted(keyvals)
    probs_real_duration = []
    for i in range(len(keyvals)):
        probs_real_duration.append(durationProbs[keyvals[i]])

    # fig_save_name = activity_name_to_compare + '_number_of_duration_probabilities_real'
    # plt.plot(keyvals, probs_real_duration)
    # plt.title('real data - number of duration probabilities')
    # # plt.show()
    # plt.savefig(fig_save_dir + fig_save_name)
    # plt.close('all')

    event_dur_prob_real_key_values_all_data_all_act.append(keyvals)
    event_dur_prob_real_all_data_all_act.append(probs_real_duration)

    excludeIndices = []
    # activitySum_real = ums.onsetProbabilityCalculator(activityDict, excludeIndices)
    activitySum_real = ums.onsetProbabilityCalculator(activityDict, excludeIndices,
                                                      idx_division_for_morning_midday_evening)
    x1 = np.sum(activitySum_real[:idx_division_for_morning_midday_evening[0]])
    x2 = np.sum(
        activitySum_real[idx_division_for_morning_midday_evening[0]:idx_division_for_morning_midday_evening[1]])
    x3 = np.sum(activitySum_real[idx_division_for_morning_midday_evening[1]:])
    # activitySum_real = ums.onsetProbabilityCalculator_V2(activityDict, excludeIndices)

    print(
        activity_name_to_compare + ' real morning: ' + str(x1) + ' afternoon: ' + str(x2) + ' evening: ' + str(x3))

    # timeOfDayProbs_real_data = ums.timeOfDayProbability(activityDict, idx_division_for_morning_midday_evening)

    # timeOfDayProbs_list = timeOfDayProbs_real_data.values()
    # timeOfDayProbs_array_real_data = np.array(list(timeOfDayProbs_list))
    # if np.sum(timeOfDayProbs_array_real_data) != 0:
    #     normalized_timeOfDayProbs_real = timeOfDayProbs_array_real_data / np.sum(timeOfDayProbs_array_real_data)
    # else:
    #     normalized_timeOfDayProbs_real = np.zeros(len(idx_division_for_morning_midday_evening))

    # fig_save_name = activity_name_to_compare + '_onset_probabilities_real'
    # plt.plot(activitySum_real)
    # plt.title('real data - onset probabilities')
    # # plt.show()
    # plt.savefig(fig_save_dir + fig_save_name)
    # plt.close('all')

    onset_prob_real_all_data_all_act.append(activitySum_real)

    pro_act_on_real, pro_act_off_real = ums.activity_daily_prob_profile_for_group_of_respondents_weights(
        oldActivityDict, weight_occ_real) 

    fig_save_name = activity_name_to_compare + '_daily_probability_profile_comparison'
  

    pd.DataFrame(pro_act_on_real).to_csv(fig_save_name +'nominority'+ '_15mins_interval.csv', header=False, index=False)
  

    print(activity_name_to_compare  + ' complete')
    

real probability on - percentage of 101c in the morning 0.02967868530782438 in the afternoon 0.01913171449595291 in the evening 0.018150600932057885 no activity 0.9818493990679421
101c real morning: 0.46953778133728447 afternoon: 0.31136517800851105 evening: 0.21909704065420452
101c complete
real probability on - percentage of 102c in the morning 0.03213146921756193 in the afternoon 0.03237674760853569 in the evening 0.0009811135638950208 no activity 0.9990188864361049
102c real morning: 0.4910881370393197 afternoon: 0.4943197983561951 evening: 0.014592064604485278
102c complete
real probability on - percentage of 103c in the morning 0.9688496443463331 in the afternoon 0.35271032622026 in the evening 0.9874908020603385 no activity 0.012509197939661517
103c real morning: 0.45485766021274554 afternoon: 0.13726392613404187 evening: 0.40787841365321265
103c complete
real probability on - percentage of 104c in the morning 0.00564140299239637 in the afternoon 0.016188373804267846 in the even

In [16]:
1

1